In [1]:
import pandas as pd 
import numpy as np


In [2]:
# PREPROCESSING
df  = pd.read_csv("results_259_Descriptive_norms-replication-XPLab-2021_group_7.csv")

# search all participants with valid responses and save their id
# vdf = valid dataframe, dataframe of valid participants
vdf = df.loc[df['response'] == 'Participants chose which action they preferred']
vdf = vdf[['submission_id']]

# vidf = valid identity dataframe; dataframe only with ID's of valid participants
vidf = pd.DataFrame()
for n in vdf['submission_id']:
    vidf = vidf.append(df.loc[df['submission_id'] == n])
    
# sdf = shortened dataframe
# every participants has 7 rows, the response value is in the third, so starting from the third row, get only every seventh
sdf = vidf.iloc[3::7, :]

# dataframe with only analytically important values
newdf = sdf[['both_norms_shown','ingroup_descriptive_norm','response']]
print(newdf)

    both_norms_shown  ingroup_descriptive_norm response
10                 1                         1        3


In [ ]:
# ANALYSIS